In [45]:
# Referencias: https://towardsdatascience.com/looking-for-a-house-build-a-web-scraper-to-help-you-5ab25badc83e 
# http://stackoverflow.com --> para consultas sobre técnicas con Python

# Se importan las librerías necesarias
from bs4 import BeautifulSoup
import string
from string import whitespace
import requests
from requests import get
from requests.adapters import HTTPAdapter
from urllib3.util import Retry
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

# Se simula que la consulta es desde un navegador real con un maximo de connect=3 para que el sitio no rechace la conexión
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
titulos = []
autores = []
editoriales = []
paginas_libros = []
puntovotos = []
puntos = []
votos = []

# Los libros están en cuatro páginas distintas del mismo sitio que se diferencian por un único número al final del link
# Se utiliza este número para iterar la dirección y recoger los libros de cada página

for page in range(0,4):
    webpage = 'https://openlibra.com/es/collection/search/category/programacion_python/pag/'+str(page)
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    # Se establece la sesión http y se descarga el contenido de la página
    session.get(webpage) 
    response = get(webpage, headers=headers)
    # Se comprueba que la página responde
    print(response)
    #Se utiliza la librería Beautiful Soup para extraer el contenido del sitio
    html_soup = BeautifulSoup(response.text, 'html.parser')
    book_containers = html_soup.find_all('div', class_="item-container")
    container = len(book_containers)
    # Se puede iterar el container que devuelve Beautiful Soup para extraer los campos que nos interesan, delimitados
    # por un marcador html diferente. Cada campo se va añadiendo al final de una lista
    for container in book_containers:
        if book_containers != []:
            # Titulo            
            titulo = container.find('h2').get_text()
            titulos.append(titulo)
            # Autor
            autor = container.find('p').get_text()
            autores.append(autor)
            # Editorial
            editorial = container.find_all('p')[1].get_text()
            editoriales.append(editorial)
            # Páginas
            paginas = container.find_all('p')[2].get_text()
            paginas_libros.append(paginas)
            # Puntovoto
            puntovoto = container.find('p', class_="rating-legend")
            puntovotos.append(puntovoto)
        else:
            break 

# Pequeño pre-procesado del resultado que se obtiene en puntovotos:
puntovotos = [str(i) for i in puntovotos]
puntovotos_final = [word[25:] for word in puntovotos]
puntovotos_final = [word[:-4] for word in puntovotos_final]

# Se crea un panda a partir de las listas resultantes del scraping        
libros = pd.DataFrame({'Título':titulos, 'Autor':autores, 'Editorial':editoriales, 'Páginas':paginas_libros, 
                      'Ratio Puntos / Votos': puntovotos_final})

# Se comprueba que el dataframe tiene buen aspecto
print (libros)

# Se vuelca el dataframe a un archivo csv de nombre Libros_Python.csv
libros.to_csv('Libros_Python.csv',index=True)      
    

<Response [404]>
<Response [200]>
<Response [200]>
<Response [200]>
                                   Título  \
0                  Solving PDEs in Python   
1          Python Notes for Professionals   
2                    Python Anti-Patterns   
3                     Intermediate Python   
4        Functional Programming in Python   
5         Introducción a la programación…   
6             Aprendiendo a Programar en…   
7   Programming and Mathematical Thinking   
8                Hands-on Python Tutorial   
9       The Coder’s Apprentice: Learning…   
10        Laboratory Manual for Computer…   
11       Python for Everybody: Exploring…   
12               Labs for Foundations of…   
13           Introducción al lenguaje de…   
14        Introducción a la programación…   
15                    Design and build a…   
16                 Make Games with Python   
17                    Python Programación   
18                 Deep Learning Tutorial   
19            Inventa tus propio